In [2]:
def generate_sequence(n):
  import random

  bases = ['a','g','c','t']
  seq = ''
  for i in range(n):
    seq += bases[random.randint(0,3)]
  return seq

def chop_sequence(sequence,min_len=3,max_len=15,min_overlap=3):
  import random

  start = 0
  end = max(random.randint(start,max_len),min_len)
  chop = [sequence[start:end]]
  while end < len(sequence):
    start = random.randint(start + 1,end - min_overlap)
    end = max(random.randint(start,start + max_len),start + min_len)
    chop += [sequence[start:end]]
  return chop

In [7]:
ref = "Fresh out of college, Barry the Bee (Jerry Seinfeld) finds the prospect of working with honey uninspiring. He flies outside the hive for the first time and talks to a human (Renée Zellweger), breaking a cardinal rule of his species. Barry learns that humans have been stealing and eating honey for centuries, and he realizes that his true calling is to obtain justice for his kind by suing humanity for theft." #generate_sequence(200)
reads = chop_sequence(ref,9,18,8)
import random
print(ref)
print(reads)
random.shuffle(reads)
seq = Sequitur(reads)
seq.construct()

Fresh out of college, Barry the Bee (Jerry Seinfeld) finds the prospect of working with honey uninspiring. He flies outside the hive for the first time and talks to a human (Renée Zellweger), breaking a cardinal rule of his species. Barry learns that humans have been stealing and eating honey for centuries, and he realizes that his true calling is to obtain justice for his kind by suing humanity for theft.
['Fresh out', 'resh out ', 'esh out of ', 'sh out of', 'h out of ', ' out of coll', ' of colle', 'of colleg', 'f college', ' college, Barry', 'lege, Bar', 'ege, Barr', 'ge, Barry the B', 'Barry the Bee (Jer', 'arry the ', 'rry the Bee (Jer', 'ry the Bee (Jer', 'he Bee (Jer', 'e Bee (Je', ' Bee (Jer', 'Bee (Jerry', 'e (Jerry ', ' (Jerry Sei', 'erry Seinfeld) f', 'infeld) find', 'feld) finds th', 'd) finds ', ') finds t', ' finds th', 'finds the', 'inds the ', 'nds the p', 'ds the prospect o', 'ospect of', 'spect of ', 'pect of working ', 'of workin', 'f working with ', 'orking with ho

'Fresh out of college, Barry the Bee (Jerry Seinfeld) finds the prospect of working with honey uninspiring. He flies outside the hive for the first time and talks to a human (Renée Zellweger), breaking a cardinal rule of his species. Barry learns that humans have been stealing and eating honey for centuries, and he realizes that his true calling is to obtain justice for his kind by suing humanity for theft.'

In [15]:
import random

results = []
for i in range(50):
    reads = chop_sequence(ref,9,18,8)
    random.getstate()
    random.shuffle(reads)
    seq = Sequitur(reads)
    results += [{
        'rand_state':random.getstate(),
        'reads':reads,
        'sequence':seq.construct()
    }]

In [35]:
import random

results = [0,0]
reads = chop_sequence(ref,9,18,8)
random.shuffle(reads)
seq = Sequitur(reads)
target = seq.construct()
for i in range(100):
    seq = Sequitur(reads)
    results[seq.construct() == target] += 1
results

[0, 100]

In [32]:
results

[0, 100]

In [39]:
import random

results = [0,0]
reads = chop_sequence(ref,9,18,8)
for i in range(100):
    random.shuffle(reads)
    seq = Sequitur(reads)
    results[seq.construct() == ref] += 1
results

[11, 89]

In [25]:
import json
with open('results.json','w') as f:
    f.write("{\n")
    for i in range(len(results)):
        f.write('"{}":{}{}\n'.format(i,json.dumps(results[i]),',' if i < len(results)-1 else ''))
    f.write("}")


In [26]:
ref = "The sly brown fox jumps swiftly over the lazy dog"
reads = chop_sequence(ref,6,11)
import random
print(ref)
print(reads)
random.shuffle(reads)

The sly brown fox jumps swiftly over the lazy dog
['The sly b', 'y brown fo', 'brown ', 'rown fox j', 'n fox jump', 'ox jumps s', 'umps s', 'mps swiftly', 'tly ov', ' over ', 'over the la', ' the lazy d', 'lazy dog']


In [42]:
from Bio import SeqIO

streptococcus_suis_reads = []
with open("database/Streptococcus_suis_reads_DRR015872_1.fastq") as file:
    for record in SeqIO.parse(file, "fastq"):
        streptococcus_suis_reads += [record.seq]
with open("database/Streptococcus_suis_reads_DRR015872_2.fastq") as file:
    for record in SeqIO.parse(file, "fastq"):
        streptococcus_suis_reads += [record.seq]
streptococcus_suis_ref = ''
with open("database\Streptococcus_suis_ref_GCF_000026745.1_ASM2674v1_genomic.fna") as file:
    for record in SeqIO.parse(file, "fasta"):
        streptococcus_suis_ref += record.seq

In [39]:
class Read:
  def __init__ (self,read,k_min=3):
    self.read = read
    self.k_min = k_min

  def __repr__(self):
    return self.read

  def __getitem__(self,key):
    return self.read[key]

  def __len__(self):
    return len(self.read)

  def partition(self,sep):
    return self.read.partition(sep)
  
  def startswith(self,key):
    return self.read.startswith(key)

  def endswith(self,key):
    return self.read.endswith(key)

  def find(self,key):
    return self.read.find(key)

  def __contains__(self,item):
    return item in self.read

  def get_all_partitions(self,root):
    index = 0
    for c in range(self.read.count(root)):
      index = self.read.find(root,index)
      part = list(self.read[index:].partition(root))
      part[0] += self.read[:index]
      yield part, index
      index += len(root)

  def connection_strength(self,other,root,index=0):
    p2 = list(other.read[other.read.find(root,index):].partition(root))
    p2[0] += other.read[:other.read.find(root,index)]
    total = max_pre = max_suf = 0
    curr_part = None
    for p1 in self.get_all_partitions(root):
      min_pre = min(len(p1[0][0]),len(p2[0]))
      min_suf = min(len(p1[0][2]),len(p2[2]))
      if min_pre == 0 and min_suf == 0: continue
      if self.read[p1[1]-min_pre:p1[1]+len(root)+min_suf] == other.read[other.read.find(root,index)-min_pre:other.read.find(root,index)+len(root)+min_suf]:
         max_pre = max(min_pre,max_pre)
         max_suf = max(min_suf,max_suf)
         if max_pre + max_suf > total:
          total = max_pre + max_suf
          curr_part = p1[0]
    return {
      'total_strength': total,
      'prefix_strength': max_pre,
      'suffix_strength': max_suf,
      'p1': curr_part,
      'p2': p2
    }

  def is_continuous_with(self,other,root,index=0):
    p2 = list(other.read[other.read.find(root,index):].partition(root))
    p2[0] += other.read[:other.read.find(root,index)]
    for p1 in self.get_all_partitions(root):
      min_pre = min(len(p1[0][0]),len(p2[0]))
      min_suf = min(len(p1[0][2]),len(p2[2]))
      # if min_pre == 0 and min_suf == 0: continue
      if self.read[p1[1]-min_pre:p1[1]+len(root)+min_suf] == other.read[other.read.find(root,index)-min_pre:other.read.find(root,index)+len(root)+min_suf]: return True
    return False

  def continues_to(self,other,root):
    return self.read.partition(root)[0].endswith(other.read.partition(root)[0])
  
  def continues_from(self,other,root):
    return self.read.partition(root)[2].startswith(other.read.partition(root)[2]) 

In [40]:
class Segment:
  def __init__(self,prefix,root,suffix,read):
    self.root = root
    self.read = read
    self.prefix = prefix
    if prefix: 
      self.prefix_strength = self.read.connection_strength(self.prefix.read,self.root)['prefix_strength']
      self.prefix.suffix_strength = self.prefix.read.connection_strength(self.read,self.prefix.root)['suffix_strength']
    else: self.prefix_strength = 0
    self.suffix = suffix
    self.suffix_strength = 0
    
  def __repr__(self):
    return '{}{}{}'.format(self.prefix.root + ' -> ' if self.prefix else '',self.root,' -> ' + self.suffix.root if self.suffix else '')

  def __hash__(self):
    return hash(self.root)

  def __eq__(self,other):
    return self.root == other.root

  def __len__(self):
    return len(self.root)

  def __getitem__(self,key):
    return self.root[key]

In [46]:
class Sequitur:
  def __init__(self,reads,k_min=3):
    self.k_min = k_min
    self.reads = map(Read,reads)
    self.transitions = {}
    self.collisions = []
    self.seq = ''
    for read in self.reads:
      seg = Segment(None,read[:k_min],None,read)
      for i in range(1,len(read)-k_min+1):
        seg.suffix = Segment(seg,read[i:i+k_min],None,read)
        if seg.root not in self.transitions: self.transitions[seg.root] = [seg]
        else:
          add = True
          pre = {
            'prefix':seg.prefix,
            'prefix_strength':seg.prefix_strength,
            'node':None
          }          
          suf = {
            'suffix':seg.suffix,
            'suffix_strength':seg.suffix_strength,
            'node':None
          }
          for node in self.transitions[seg.root]:
            curr = node
            if curr.read.endswith(seg.root) and seg.read.startswith(seg.root):
              while curr and curr.read.endswith(seg.root) and seg.read.startswith(seg.root):
                curr = curr.suffix
            if curr:
              conn = curr.read.connection_strength(seg.read,seg.root,i-1)
              if conn['total_strength'] > 0:
                add = False
                if node.prefix_strength < seg.prefix_strength and (conn['prefix_strength'] > 0 or conn['suffix_strength'] >= 3):
                  pre['prefix'] = seg.prefix
                  pre['prefix_strength'] = seg.prefix_strength
                  pre['node'] = node 
                if node.suffix_strength < seg.suffix_strength and (conn['suffix_strength'] > 0 or conn['prefix_strength'] >= 3):
                  suf['suffix'] = seg.suffix
                  suf['suffix_strength'] = seg.suffix_strength
                  suf['node'] = node 
            else:
              add = False
              self.collisions += [seg]
          if add: self.transitions[seg.root] += [seg]
          else:
            if pre['node']:
              pre['node'].prefix = pre['prefix']
              pre['node'].prefix_strength = pre['prefix_strength']
              seg.prefix.suffix = pre['node']
            if suf['node']:
              suf['node'].suffix = suf['suffix']
              suf['node'].suffix_strength = suf['suffix_strength']
              seg.suffix.prefix = suf['node']
        seg = seg.suffix
      if seg.root not in self.transitions: self.transitions[seg.root] = [seg]
      else:
        add = True
        pre = {
          'prefix':seg.prefix,
          'prefix_strength':seg.prefix_strength,
          'node':None
        } 
        for node in self.transitions[seg.root]:
          curr = node
          if curr.read.startswith(seg.root) and seg.read.endswith(seg.root):
            while curr and curr.read.startswith(seg.root) and seg.read.endswith(seg.root):
              curr = curr.prefix
          if curr:
            conn = curr.read.connection_strength(seg.read,seg.root,i-1)
            if conn['total_strength'] > 0:
              add = False
              if node.prefix_strength < seg.prefix_strength and (conn['prefix_strength'] > 0 or conn['suffix_strength'] >= 3):
                pre['prefix'] = seg.prefix
                pre['prefix_strength'] = seg.prefix_strength
                pre['node'] = node
          else:
            add = False
            self.collisions += [seg]
        if add: self.transitions[seg.root] += [seg]
        else:
          if pre['node']:
            pre['node'].prefix = pre['prefix']
            pre['node'].prefix_strength = pre['prefix_strength']
            seg.prefix.suffix = pre['node']
      
  def __repr__(self):
    return str(self.transition)

  def construct(self):
    import random

    seed = curr = random.choice(self.transitions[random.choice(list(self.transitions.keys()))])
    self.seq = seed.root
    n = 0
    while curr.suffix and n < 400:
      curr = curr.suffix
      self.seq = self.seq + curr.root[-1]
      n += 1
    curr = seed
    n = 0
    while curr.prefix and n < 400:
      curr = curr.prefix
      self.seq = curr.root[0] + self.seq
      n += 1
    return self.seq

In [47]:
streptococcus_suis_reads = list(map(str,streptococcus_suis_reads))
seq = Sequitur(streptococcus_suis_reads,k_min=26)

In [44]:
seq.transitions

NameError: name 'seq' is not defined

In [ ]:
seq.construct()

In [38]:
streptococcus_suis_reads

['...',
 ...]